In [1]:
import pandas as pd
import numpy as np
import os
import sys

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [2]:
cwd = os.getcwd()

education_folder_path = os.path.dirname(cwd)

sys.path.append(education_folder_path)

In [3]:
from global_functions import remove_spaces_in_columns, up_load_df

In [4]:
#בתי ספר וגנים מעיריית ירושלים
JLM=up_load_df(r'{}\background_files\jerusalem_muni'.format(education_folder_path),'מוסדות בירושלים 2020')
JLM=remove_spaces_in_columns(JLM)

In [5]:
#קאורדינטות של מוסדות חינוך במרחב ירושלים
JTMT_setls_schools_coordinates_with_src=up_load_df(r'{}\background_files'.format(education_folder_path),'JTMT_setls_schools_coordinates_with_src')
JTMT_setls_schools_coordinates_with_src=remove_spaces_in_columns(JTMT_setls_schools_coordinates_with_src)

In [6]:
JLM.rename(columns={'סמל_חינוך': 'סמל_מוסד'}, inplace=True)

JLM = JLM.drop(columns=['פיקוח', 'מעמד_משפטי', "מס'_תלמידים_ז-ט", "מס'_תלמידים_י-יד", 'קוד_אזור_סטיסטי', 'תאור_אזור_סטטיסטי', 'Unnamed:_18', 'Unnamed:_19'])

JLM=JLM[JLM['שלב_חינוך'] != 'גני ילדים']

In [7]:
JLM.to_excel('JLM_without_garden.xlsx', index=False)

In [ ]:
# סינון שורות שבהן העמודה "סה"כ" אינה שווה לאפס
JLM=JLM[JLM['סה"כ_תלמידים'] != 0]

In [ ]:
JTMT_setls_schools_coordinates_with_src.head()

In [ ]:
is_unique = JTMT_setls_schools_coordinates_with_src['סמל_מוסד'].is_unique
is_unique

In [ ]:
rows_with_zero = JLM[JLM['coordinate_x'] == 0]
JLM = JLM[JLM['coordinate_x'] != 0]

In [ ]:
JLM['SRC'] = 'jerusalem_muni'

In [ ]:
rows_with_zero['SRC'] = ''

In [ ]:
# מיזוג הטבלאות לפי העמודה "סמל_מוסד"
rows_with_zero = pd.merge(rows_with_zero, JTMT_setls_schools_coordinates_with_src[['סמל_מוסד', 'coordinate_x', 'coordinate_y', 'SRC']],
                     on='סמל_מוסד', how='left')

# עדכון העמודות המקוריות
rows_with_zero['coordinate_x'] = rows_with_zero['coordinate_x_y']
rows_with_zero['coordinate_y'] = rows_with_zero['coordinate_y_y']
rows_with_zero['SRC'] = rows_with_zero['SRC_y']

# מחיקת העמודות המיותרות
rows_with_zero = rows_with_zero.drop(['coordinate_x_x', 'coordinate_y_x', 'coordinate_x_y', 'coordinate_y_y', 'SRC_x', 'SRC_y'], axis=1)
rows_with_zero.head()

In [ ]:
# מיזוג הטבלאות
JLM=pd.concat([JLM, rows_with_zero], ignore_index=True)

In [ ]:
JLM['סמל_מוסד'] = JLM['סמל_עירייה']
JLM.drop(columns='סמל_עירייה', inplace=True)

In [ ]:
is_unique = JLM['סמל_מוסד'].is_unique
is_unique

In [ ]:
JLM.head()

In [ ]:
JLM.to_excel('JLM_1.xlsx', index=False)

In [ ]:
# הגדרת רשימת הכיתות לטווח א' עד ח'
class_map = ['א', 'ב', 'ג', 'ד', 'ה', 'ו', 'ז', 'ח', 'ט', 'י', 'יא', 'יב', 'יג', 'יד']

# הגדרת עמודות לכיתות עם NaN כערך ברירת מחדל
for single_class in class_map:
    JLM[f'כיתה_{single_class}'] = np.nan

# פונקציה לחישוב מספר התלמידים לכל כיתה בטווח
def first_15_division(row):
    start_idx = class_map.index(row['מכיתה'])
    end_idx = class_map.index(row['עד_כיתה'])
    class_range = class_map[start_idx:end_idx + 1]

    students_remaining = row['סה"כ_תלמידים']
    for single_class in class_range:
        students_in_class = min(15, students_remaining)
        row[f'כיתה_{single_class}'] = students_in_class
        students_remaining -= students_in_class

    row['סה"כ_תלמידים'] = students_remaining
    return row


# החלת הפונקציה על כל השורות בטבלה
JLM = JLM.apply(first_15_division, axis=1)

# הצגת הטבלה החדשה
JLM.head()

In [ ]:
# פונקציה לחלוקת התלמידים הנותרים שווה בשווה
def equal_distribution_students(row):
    # איתור הטווח של הכיתות
    start_idx = class_map.index(row['מכיתה'])
    end_idx = class_map.index(row['עד_כיתה'])
    class_range = class_map[start_idx:end_idx + 1]
    
    # חישוב מספר התלמידים הנותרים
    students_remaining = row['סה"כ_תלמידים']
    num_of_classes = len(class_range)
    
    if num_of_classes > 0:
        # חלוקת התלמידים הנותרים שווה בשווה לכיתות בטווח
        students_per_class = students_remaining / num_of_classes
        for single_class in class_range:
            # אם יש כבר ערך בעמודת הכיתה, נוסיף את הערך החדש
            if pd.notna(row[f'כיתה_{single_class}']):
                row[f'כיתה_{single_class}'] += students_per_class
            else:
                row[f'כיתה_{single_class}'] = students_per_class

    # עדכון סה"כ תלמידים לאחר חלוקה
    row['סה"כ_תלמידים'] = 0
    return row

# החלת הפונקציה על כל השורות בטבלה
JLM = JLM.apply(equal_distribution_students, axis=1)

# הצגת הטבלה החדשה
JLM.head()

In [ ]:
class_map = ['כיתה_א', 'כיתה_ב', 'כיתה_ג', 'כיתה_ד', 'כיתה_ה', 'כיתה_ו', 'כיתה_ז', 'כיתה_ח', 'כיתה_ט', 'כיתה_י', 'כיתה_יא', 'כיתה_יב', 'כיתה_יג', 'כיתה_יד']

# בדיקה ועדכון כל תא אם מספר התלמידים קטן מ-15
for single_class in class_map:
    JLM[single_class] = JLM[single_class].apply(lambda x: x if pd.isna(x) or x >= 15 else 0)

# הצגת הטבלה המעודכנת
JLM.head()

In [ ]:
# חלוקה לשכבות גיל
ages = {
    'ele': ['כיתה_א', 'כיתה_ב', 'כיתה_ג', 'כיתה_ד', 'כיתה_ה', 'כיתה_ו'],
    'mid': ['כיתה_ז', 'כיתה_ח', 'כיתה_ט'],
    'high': ['כיתה_י', 'כיתה_יא', 'כיתה_יב', 'כיתה_יג', 'כיתה_יד']
}

# חישוב מספר התלמידים לכל שכבה
JLM['ele_stu'] = JLM[ages['ele']].sum(axis=1)
JLM['mid_stu'] = JLM[ages['mid']].sum(axis=1)
JLM['high_stu'] = JLM[ages['high']].sum(axis=1)

#sector
JLM.loc[JLM['אגף']=='ערבי','sector'] = 1
JLM.loc[JLM['אגף']=='חרדי','sector'] = 2
JLM.loc[JLM['אגף']=='כללי','sector'] = 3

JLM_final = JLM.drop(['אגף', 'שלב_חינוך', 'כיתה_א', 'כיתה_ב', 'כיתה_ג', 'כיתה_ד', 'כיתה_ה', 'כיתה_ו', 'כיתה_ז', 'כיתה_ח', 'כיתה_ט', 'כיתה_י', 'כיתה_יא', 'כיתה_יב', 'כיתה_יג', 'כיתה_יד', 'מספר_כיתות', 'סה"כ_תלמידים', 'מכיתה', 'עד_כיתה', 'שם_מוסד', 'סוג_חינוך'], axis=1)

JLM_final.rename(columns={'סמל_מוסד': 'ID'}, inplace=True)

JLM_final.rename(columns={'coordinate_x': 'x'}, inplace=True)

JLM_final.rename(columns={'coordinate_y': 'y'}, inplace=True)

In [ ]:
JLM.to_excel(r'{}\JLM.xlsx'.format(cwd), index=False)

In [ ]:
JLM.head()

In [ ]:
JLM_final.head()

In [ ]:
JLM_final.to_excel('JLM_final_1.xlsx', index=False)